## Вариант 7
## Работу выполнил студент группы А482 Мировский Н.С.

<div><img src="./lol.png" width="400"/></div>

# Функция <em><strong>solve</strong></em> принимает следующие аргументы
##### k1,k2,k3,k4 - жесткости пружин
##### c1,c2,c3,c3,c4 - коэффициенты фемпфирования
##### m1,m2,m3 - массы тел
##### a1,a2 - расстояния от пружин до центра масс 2 тела по горизонтали
##### a3,a4 - расстояния от пружин до центра масс 3 тела по горизонтали
##### h1,h2,h3,h4 - расстояния до верхних пружин от центра масс 1 тела по горизонтали
##### l1,l2,l3,l4,l5 - расстояния до нижних амортизаторов от центра масс 1 тела по горизонтали
##### t5,t6,t7,t8,t9 - время между удароми каждого амортизатора в период времени с (1,3) с шагом 0.002
#####  I1,I2,I3 - моменты инерции тел

### А также имеет следующи константы:
##### delta_t=0.002
##### gamma = 0.5
##### betta = 0.25

In [ ]:
import numpy as np

In [ ]:
#Функция для создания синусоиды
def sinys(t,amplityda):
    return amplityda*np.sin(np.pi*t)

def solve(k1,k2,k3,k4,c1,c2,c3,c4,m1,m2,m3,a1,a2,a3,a4,h1,h2,h3,h4,l1,l2,l3,l4,l5,t5,t6,t7,t8,t9,I1,I2,I3,delta_t=0.002,gamma = 0.5,betta = 0.25):
    #Создаем начальные вектора перемещений скоростей и ускорений
    X = np.zeros((6,1))
    X_t = np.zeros((6,1))
    X_tt = np.zeros((6,1))
    #Создаем диагональную матрицу
    K_diag = np.diag([k1,k1,k2,k2,k3,k3,k4,k4,k4])
    D = np.array([[-1,h1,1,-a1,0,0],
    [-1,h2,1,a2,0,0],
    [-1,-h3,0,0,1,-a3],
    [-1,-h4,0,0,1,a4],
    [1,-l1,0,0,0,0],
    [1,-l3,0,0,0,0],
    [1,l5,0,0,0,0],
    [1,l4,0,0,0,0],
    [1,l2,0,0,0,0]])
    K = np.transpose(D)@K_diag@D
    #
    C_diag = np.diag([0,0,0,0,c3,c3,c4,c4,c4])
    C = np.array([[0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [1,-l1,0,0,0,0],
    [1,-l3,0,0,0,0],
    [1,l5,0,0,0,0],
    [1,l4,0,0,0,0],
    [1,l2,0,0,0,0]])
    C = np.transpose(C)@C_diag@C
    #Создаем диагональную матрицу масс и моментов
    M = np.diag([m1,I1,m2,I2,m3,I3])
    #Делаем матрицу внешних воздействий
    n = 5
    y_voz = np.zeros((10,1001))
    y_voz[0,:] = [i*0.001 for i in range(1000,3001,2)]
    index_t = [np.where(y_voz == t5)[-1][-1],np.where(y_voz == t6)[-1][-1],np.where(y_voz == t7)[-1][-1],np.where(y_voz == t8)[-1][-1],np.where(y_voz == t9)[-1][-1]]
    for i in index_t:
        y_voz[n,i:] =  np.concatenate([np.array([sinys(r,1) for r in np.linspace(0,1,75)]),np.zeros((1001-75-i))])
        n+=1
    y_impact = y_voz[1:,1:]
    #Создаем обратную матрицу
    M_reverse = np.linalg.inv(M+delta_t*gamma*C+delta_t**2*betta*K)
    for i in range(999):
        #Формируем матрицы 
        #Ускорений
        F = np.transpose(D)@K_diag@y_impact[:,i+1]-C@(X_t[:,-1]+delta_t*(1-gamma)*X_tt[:,-1])-K@(X[:,-1]+delta_t*X_t[:,-1]+1/2*delta_t**2*(1-2*betta)*X_tt[:,-1])
        X_tt = np.concatenate([X_tt,(np.dot(F,M_reverse)).reshape(-1,1)],axis =1)
        #Скоростей
        X_tn = X_t[:,i]+delta_t*gamma**2*X_tt[:,-1]+(1-gamma)*X_tt[:,i]
        X_t = np.concatenate([X_t,X_tn.reshape(-1,1)],axis =1)
        #Перемещений
        X_n = X[:,i]+delta_t*X_t[:,i]+(1/2)*(delta_t**2)*2*betta*X_tt[:,-1]+(1-2*betta)*X_tt[:,i]
        X = np.concatenate([X,X_n.reshape(-1,1)],axis =1)
    #return X,X_t,X_tt
    #Если нужен вывод матриц ускорений,скоростей и перемещений нужно убрать # перед return